

<h1 align=center><font size = 5>Exploratory Data Analysis with SQL</font></h1>


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.


Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 81.2 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=e436b9e9da2ba46db0d5d37f6e070d2f76875719df062a0c99c72d1f2e77789a
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas==1.1.5

In [ ]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [ ]:
import pandas as pd
df = pd.read_csv("data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


**Note:This below code is added to remove blank rows from table**


In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

## Unique launch sites names in the space mission


In [ ]:
%sql select distinct(Launch_Site) from SPACEXTBL

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



## Records of launch sites that starts with 'CCA'


In [ ]:
%%sql select * from SPACEXTBL
    where Launch_Site LIKE "CCA%" limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## Total payload mass carried by boosters launched by NASA (CRS)


In [ ]:
%%sql select sum(PAYLOAD_MASS__KG_) as "Payload Carried by NASA (CRS)" from SPACEXTBL
    where Customer = "NASA (CRS)"

 * sqlite:///my_data1.db
Done.


Payload Carried by NASA (CRS)
45596


## Average payload mass carried by booster version F9 v1.1


In [ ]:
%%sql select avg(PAYLOAD_MASS__KG_) as "Payload Avg by F9 v1.1" from SPACEXTBL
    where Booster_Version like "F9 v1.1"

 * sqlite:///my_data1.db
Done.


Payload Avg by F9 v1.1
2928.4


## List of dates when the first succesful landing outcome in ground pad was acheived.


In [ ]:
%%sql select min(Date) as Date, landing_outcome from SPACEXTBL
    where landing_outcome = "Success (ground pad)"

 * sqlite:///my_data1.db
Done.


Date,Landing_Outcome
2015-12-22,Success (ground pad)


## Names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [ ]:
%%sql select Booster_Version, PAYLOAD_MASS__KG_, landing_outcome from SPACEXTBL
    where PAYLOAD_MASS__KG_ between 4000 and 6000
    and landing_outcome like "Success (drone ship)"

 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_,Landing_Outcome
F9 FT B1022,4696,Success (drone ship)
F9 FT B1026,4600,Success (drone ship)
F9 FT B1021.2,5300,Success (drone ship)
F9 FT B1031.2,5200,Success (drone ship)


## Total number of successful and failure mission outcomes


In [ ]:
%%sql select landing_outcome, count(*) as count from SPACEXTBL
    where landing_outcome like "Success%" or landing_outcome like "Failure%"
    group by landing_outcome

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count
Failure,3
Failure (drone ship),5
Failure (parachute),2
Success,38
Success (drone ship),14
Success (ground pad),9


## Names of the booster versions which have carried the maximum payload mass.


In [ ]:
%%sql select distinct(Booster_Version), PAYLOAD_MASS__KG_ as Max_Payload from SPACEXTBL
    where PAYLOAD_MASS__KG_ = (select MAX(PAYLOAD_MASS__KG_) from SPACEXTBL)

 * sqlite:///my_data1.db
Done.


Booster_Version,Max_Payload
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


### Record of Month names, Failure landing outcomes in drone ship, Booster versions, Launch site for the year 2015.


In [ ]:
%%sql select case
    when substr(Date, 6, 2) = '01' then 'January'
    when substr(Date, 6, 2) = '02' then 'February'
    when substr(Date, 6, 2) = '03' then 'March'
    when substr(Date, 6, 2) = '04' then 'April'
    when substr(Date, 6, 2) = '05' then 'May'
    when substr(Date, 6, 2) = '06' then 'June'
    when substr(Date, 6, 2) = '07' then 'July'
    when substr(Date, 6, 2) = '08' then 'August'
    when substr(Date, 6, 2) = '09' then 'September'
    when substr(Date, 6, 2) = '10' then 'October'
    when substr(Date, 6, 2) = '11' then 'November'
    when substr(Date, 6, 2) = '12' then 'December'
    else 'Unknown'
  end as Month,
  landing_outcome, booster_version, launch_site from SPACEXTBL
    where landing_outcome like "Failure (drone ship)"
    and substr(Date, 1, 4) = '2015';

 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
October,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


## Count of landing outomes between 2010-06-04 and 2017-03-20 for drone ship failure and ground pad success


In [ ]:
%%sql select landing_outcome, count(*) as Count FROM SPACEXTBL
    where Date between '2010-06-04' and '2017-03-20'
    and landing_outcome = 'Failure (drone ship)' OR landing_outcome = 'Success (ground pad)'
    group by landing_outcome
    order by Count desc

 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
Success (ground pad),9
Failure (drone ship),5
